In [1]:
import bokeh.plotting as bp
from bokeh.models import HoverTool
from bokeh.plotting import save

import inspect
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import os
from numpy import linalg
from numpy.linalg import norm
from scipy.spatial.distance import squareform, pdist
import nltk
from os import listdir
import re
from sklearn.decomposition import LatentDirichletAllocation



# Importing sklearn and TSNE.
import sklearn
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
from sklearn.preprocessing import scale
from os.path import isfile, join


# We'll hack a bit with the t-SNE code in sklearn.
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.manifold.t_sne import (_joint_probabilities,
                                    _kl_divergence)
#from sklearn.utils.extmath import _ravel
# Random state we define this random state to use this value in TSNE which is a randmized algo.
RS = 25111993

# Importing matplotlib for graphics.
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib
%matplotlib inline

import tqdm
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import TweetTokenizer
import json
from string import punctuation

import string 
from bs4 import BeautifulSoup

# Importing seaborn to make nice plots.
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
rc={"lines.linewidth": 2.5})

In [2]:
###FUNZIONI NECESSARIE PER ESTRAZIONE DATI DAGLI ARTICOLI PER POI CREARE LA TERM DOCUMENT MATRIX

def news_analyzer(news, tokenizer, stemmer, stop_words):
    # ============== YOUR CODE HERE ==============
    campi_interesse = ['testo','titolo_articolo','sottotitolo']
    for campo in campi_interesse:
        #apostrofi fix
        news[campo] = BeautifulSoup(news[campo], "lxml").get_text()
        news[campo] = news[campo].replace("'"," ")
        news[campo] = re.sub("http\S+", " link ", news[campo])
        news[campo] = tokenizer.tokenize(news[campo])
        news[campo] = [token for token in news[campo] if not token.isdigit() and token not in stop_words]
        #news[campo] = [stemmer.stem(token) for token in news[campo] if not token.isdigit() and token not in stop_words]
    # ============================================

    return news

def preproc(news_dir):
    nltk.download("stopwords")
    stop_words = stopwords.words('italian') + list(punctuation)
    stemmer = SnowballStemmer("italian")
    tokenizer = TweetTokenizer(
        preserve_case=False,
        strip_handles=True
    )
    X_train =  []
    onlyfiles = [f for f in listdir(news_dir) if isfile(join(news_dir, f))]
    for articolo in onlyfiles:
        docJson = open(news_dir+articolo,"r")
        jsonData = json.loads(docJson.read())

        X_train.append(jsonData)
        docJson.close()

    return [news_analyzer(news, tokenizer, stemmer, stop_words) for news in tqdm.tqdm(X_train)]

# An user defined function to create scatter plot of vectors
def scatter(x, colors):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 8))

    # We create a scatter plot.
    f = plt.figure(figsize=(32, 32))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=120,
                    c=palette[colors.astype(np.int)])
    #plt.xlim(-25, 25)
    #plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each cluster.
    txts = []
    for i in range(18):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=50)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts
def distribuzione_frequenze(tf):
    tdf = np.sum(tf.toarray(), axis=0)
    df ={}
    for i in tdf:
        if i in df:
            df[i] += 1
        else:
            df[i] = 1

    k = sorted(df.keys())
    freq = [k,[]]
    for x in freq[0]:
        freq[1].append(df[x])
    return (freq)


In [3]:
economia = preproc("C:/Users/Cristy/Documents/Universita/big_data/articoli_economia/")
cultura = preproc("C:/Users/Cristy/Documents/Universita/big_data/articoli_cultura/")
tech = preproc("C:/Users/Cristy/Documents/Universita/big_data/articoli_tech/")
politica = preproc("C:/Users/Cristy/Documents/Universita/big_data/articoli_politica/")
for articolo in economia:
    articolo['categoria'] = "Economia"
for articolo in cultura:
    articolo['categoria'] = "Cultura"
for articolo in tech:
    articolo['categoria'] = "Tech"
for articolo in politica:
    articolo['categoria'] = "Politica"
dati_preprocessati =  tech + politica + cultura + economia

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 400/400 [00:00<00:00, 498.76it/s]
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 400/400 [00:00<00:00, 425.26it/s]
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 400/400 [00:00<00:00, 427.99it/s]
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Cristy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
100%|██████████| 400/400 [00:01<00:00, 351.94it/s]


In [5]:
#####Costruisci la term document matrix e poi cerca
documents = [' '.join([word for word in x['testo']] + x['tags'] + x['sottotitolo'] + x['titolo_articolo']) for x in dati_preprocessati if x['categoria']!= "Cultura & Spettacoli"]
labels = np.array([x['categoria'] for x in dati_preprocessati])
#print documents
no_features = 300000
tf_vectorizer = CountVectorizer(ngram_range=(1,12), min_df=14, max_df=70,encoding='utf-8',max_features=no_features, lowercase=True)
tf = tf_vectorizer.fit_transform(documents)
#create dataFrame
df = pd.DataFrame(tf.toarray().transpose(), index = tf_vectorizer.get_feature_names())

print(distribuzione_frequenze(tf))
#'lda__n_components': 6, 'lda__learning_decay': 0.7, 'count_mx__ngram_range': (1, 12), 'count_mx__min_df': 14, 'count_mx__max_df': 70

[[15, 16, 17, 20, 21, 22, 23, 25, 26, 27, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 95, 96, 97, 98, 100, 102, 103, 104, 105, 106, 107, 110, 115, 116, 117, 118, 120, 121, 124, 125, 127, 128, 130, 138, 140, 142, 145, 150, 151, 152, 157, 160, 193, 257], [769, 136, 8, 524, 102, 10, 1, 358, 60, 17, 4, 236, 56, 15, 2, 1, 181, 39, 12, 5, 126, 32, 15, 2, 101, 27, 13, 3, 2, 82, 21, 5, 3, 60, 25, 7, 1, 4, 72, 16, 6, 5, 45, 17, 7, 3, 5, 38, 17, 7, 1, 32, 5, 3, 2, 2, 19, 6, 6, 3, 2, 18, 5, 2, 1, 1, 10, 7, 7, 1, 2, 2, 5, 1, 1, 2, 3, 1, 1, 6, 4, 1, 2, 1, 2, 1, 1, 3, 3, 1, 4, 1, 2, 1, 1, 1, 1, 3, 1, 1, 1, 1]]


In [6]:
print(df.shape)
print(len(documents))


(3497, 1600)
1600


In [8]:
###Creamo i cluster con LDA

no_topics = 6
# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics,max_iter=100,learning_decay=0.7, learning_method='online',random_state=0).fit(tf)

#'lda__n_components': 6, 'lda__learning_decay': 0.7, 'count_mx__ngram_range': (1, 12), 'count_mx__min_df': 14, 'count_mx__max_df': 70

In [9]:
X_topics = lda.fit_transform(tf)
#Correction
threshold = 0.1
_idx = np.amax(X_topics, axis=1) > threshold  # idx of doc that above the threshold
X_topics = X_topics[_idx]
X_topics.shape

(1600, 6)

In [10]:
# Fit the model using t-SNE randomized algorithm
tsne_model = TSNE(n_components=3, verbose=1, random_state=0, init='pca')

In [11]:
# N-D -> 2-D
tsne_lda = tsne_model.fit_transform(X_topics)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1600 samples in 0.000s...
[t-SNE] Computed neighbors for 1600 samples in 0.047s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1600
[t-SNE] Computed conditional probabilities for sample 1600 / 1600
[t-SNE] Mean sigma: 0.002335
[t-SNE] KL divergence after 250 iterations with early exaggeration: 51.630032
[t-SNE] KL divergence after 1000 iterations: -1.186330


In [12]:
_topics = X_topics
len(_topics)

1600

In [13]:
# 20 colors
colormap = np.array([
    "#1f77b4", "#bbc7e8", "#cc7f0e", "#ddbb78"
])

In [14]:
#Then we find the most likely topic for each news:
_lda_keys = []
for i in range(X_topics.shape[0]):
  _lda_keys +=  _topics[i].argmax(),

len(_lda_keys)


1600

In [15]:
#and get top words for each topic:
n_top_words = 5
topic_summaries = []
topic_word = lda.components_  # all topic words
vocab = tf_vectorizer.get_feature_names()
for i, topic_dist in enumerate(topic_word):
  topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1] # get!
  topic_summaries.append(' '.join(topic_words)) # append!


In [16]:
from plotly.plotly import iplot
import plotly as pl
from plotly.graph_objs import Scatter3d, Data, Marker

# First three dimensions from reduced X VS the Y
walkers = []

trace0 = Scatter3d(
    x=tsne_lda[0:199, 0],
    y=tsne_lda[0:199, 1],
    z=tsne_lda[0:199, 2], 
    marker=Marker(color=colormap[0], colorscale='Portland'),
    mode='markers'
)

trace1 = Scatter3d(
    x=tsne_lda[200:399, 0],
    y=tsne_lda[200:399, 1],
    z=tsne_lda[200:399, 2], 
    marker=Marker(color=colormap[1], colorscale='Portland'),
    mode='markers'
)

trace2 = Scatter3d(
    x=tsne_lda[400:599, 0],
    y=tsne_lda[400:599, 1],
    z=tsne_lda[400:599, 2], 
    marker=Marker(color=colormap[2], colorscale='Portland'),
    mode='markers'
)

trace3 = Scatter3d(
    x=tsne_lda[600:799, 0],
    y=tsne_lda[600:799, 1],
    z=tsne_lda[600:799, 2], 
    marker=Marker(color=colormap[3], colorscale='Portland'),
    mode='markers'
)

walkers.append(trace0)
walkers.append(trace1)
walkers.append(trace2)
walkers.append(trace3)

data = Data(walkers) 
pl.tools.set_credentials_file(username='cristi.gutzu', api_key='S4SFAPdXM3dUxDF0wmxT')
iplot(data, filename = 'pca-cloud')


C:\Users\Cristy\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:426: DeprecationWarning:

plotly.graph_objs.Marker is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Marker
  - plotly.graph_objs.histogram.selected.Marker
  - etc.


C:\Users\Cristy\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:39: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.


C:\Users\Cristy\Anaconda3\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [36]:
## title = '1024 articoli ANSA'
from bokeh.plotting import figure 
from bokeh.transform import factor_cmap
from bokeh.io import output_notebook, show

print(len(tsne_lda[:,0]))

cats = [x['categoria'] for x in dati_preprocessati]
testi = [x['testo'] for x in dati_preprocessati]
data = { 'x' : tsne_lda[:,0],
         'y' :  tsne_lda[:,1],
         'z' :  tsne_lda[:,2],
        'content': cats, 
        'topic_key': [str(x) for x in cats]}
 
source = bp.ColumnDataSource(data)

p = figure(plot_width=1100, plot_height=1100, tooltips="content: @content - topic: @topic_key")
index_cmap=factor_cmap('topic_key', palette=colormap, factors=list(set([str(x) for x in cats])) )
p.surface3d('x', 'y','z', size=10, source=source, fill_color=index_cmap)


show(p)
list(set([str(x) for x in cats]))


799


AttributeError: 'Figure' object has no attribute 'surface3d'

In [ ]:
#Vogliamo un metodo per ottimizare il parametro del countVectorizer.
#che e' ngarm range, questo parametro vive in N.

from collections import Counter
distr = {"Tech":[], "Politica":[], "Economia":[], "Cultura":[]}
i=0
for idex, doc in enumerate(dati_preprocessati):
    topic = (_lda_keys[i])
    distr[doc['categoria']].append(topic)
    i=i+1
def CountFrequency(my_list): 
  
    # Creating an empty dictionary  
    freq = {} 
    for item in my_list: 
        if (item in freq): 
            freq[item] += 1
        else: 
            freq[item] = 1
    lista = [item[1] for item in freq.items()]
    for key, value in freq.items(): 
        print ("% d : % d"%(key, value)) 
        
    return (max(lista)/sum(lista))
    
c1=CountFrequency(distr["Tech"])
print("#######")
c2=CountFrequency(distr["Economia"])
print("#######")

c3=CountFrequency(distr["Politica"])
print("#######")

c4=CountFrequency(distr["Cultura"])

indice = (c1+c2+c3+c4)/4
print(indice)

In [ ]:


# randomly choose a news (within a topic) coordinate as the crucial words coordinate
topic_coord = np.empty((X_topics.shape[1], 2)) * np.nan
for topic_num in _lda_keys:
  if not np.isnan(topic_coord).any():
    break
  topic_coord[topic_num] = tsne_lda[_lda_keys.index(topic_num)]

# plot crucial words
for i in range(X_topics.shape[1]):
  plot_lda.text(topic_coord[i, 0], topic_coord[i, 1], [topic_summaries[i]])

# hover tools
hover = plot_lda.select(dict(type=HoverTool))
hover.tooltips = {"content": "@content - topic: @topic_key"}

# save the plot
save(plot_lda, '{}.html'.format(title))


In [ ]:
help(bp.ColumnDataSource)